In [9]:
from bs4 import BeautifulSoup
import urllib3

In [15]:
http = urllib3.PoolManager()
business_resp = http.request("GET", "http://inshorts.com/en/read/business")

C:\Users\mateu\AppData\Local\Programs\Python\Python36\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [18]:
business_soup = BeautifulSoup(business_resp.data, 'html.parser')

In [45]:
unflatten = [i.contents for i in business_soup.findAll("div", {"itemprop": 'articleBody'})]
business_stories = [item for sublist in unflatten for item in sublist]

In [84]:
import json


def extract_inshorts(categories, min_items):
    result = []
    for category in categories:
        labeled = []
        offset = ''
        while len(labeled) < min_items:
            downloaded = download_for(category, offset)
            offset = downloaded['offset']
            labeled += downloaded['stories']
        result += labeled
    return result


def download_for(category, offset):
    http = urllib3.PoolManager()
    params = {'category': category, 'news_offset': offset}
    resp = http.request('POST', 'https://inshorts.com/en/ajax/more_news', body=json.dumps(params))
    obj = json.loads(resp.data)
    return {'stories': label(parse_stories_from(obj['html']), category), 'offset': obj['min_news_id']}


def label(stories, category):
    return [{"label": category, "content": story} for story in stories]


def parse_stories_from(html):
    soup = BeautifulSoup(html, 'html.parser')
    unflattened = [i.contents for i in soup.findAll("div", {"itemprop": 'articleBody'})]
    stories_list = [item for sublist in unflattened for item in sublist]
    return stories_list

In [85]:
df = pd.DataFrame(extract_inshorts(['business', 'sports'], 100))

C:\Users\mateu\AppData\Local\Programs\Python\Python36\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


C:\Users\mateu\AppData\Local\Programs\Python\Python36\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


C:\Users\mateu\AppData\Local\Programs\Python\Python36\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


C:\Users\mateu\AppData\Local\Programs\Python\Python36\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


C:\Users\mateu\AppData\Local\Programs\Python\Python36\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


C:\Users\mateu\AppData\Local\Programs\Python\Python36\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


C:\Users\mateu\AppData\Local\Programs\Python\Python36\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


C:\Users\mateu\AppData\Local\Programs\Python\Python36\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [91]:
df['label'].describe()

count        200
unique         2
top       sports
freq         100
Name: label, dtype: object